In [18]:
# https://pypi.org/project/exif/
# pip install exif

# https://pypi.org/project/pymediainfo/
# pip install pymediainfo

In [21]:
import pandas as pd
# import PIL.Image
from exif import Image
from pprint import pprint
from pymediainfo import MediaInfo
from pathlib import Path
import numpy as np
import os
import re
import numpy as np
import json
from shutil import copy2

In [2]:
path = r'G:\Google Photo_extr\Takeout\Google Фото'
# path = Path(r'G:\Google Photo_extr\Takeout\Google Фото')
# path = Path(r'D:\Рабочая\!_Scripts\change_file_creation_time')
# path = Path('G:/temp/Тест фото')
# path = Path('E:\Тест фото')

In [18]:
def get_file_table(dir_path):
    df = pd.DataFrame(columns=['name', 'extension', 'path'])
    path = Path(dir_path)
    for file in path.glob('**/*.*'):
        if file.is_file():
            name = file.stem
            extension = file.suffix.lower()
            full_path = file
            df.loc[len(df)] = [name, extension, full_path]
    return df

In [33]:
def get_photo_exif_date(full_path):
    """
    функция получает на вход полный путь к файлу картинки и возвращает кортеж из трёх дат получинных из exif:
        (datetime, datetime_original, datetime_digitized)
    """
    try:
        with open(full_path, 'rb') as image_file:
            my_image = Image(image_file)
    except:
        return np.nan
    # паттерн для перевода дат в нужный формат (дата) (время)
    fpattern = re.compile('(\d\d\d\d:\d\d:\d\d) (\d\d:\d\d:\d\d)') 
    
    # считываем 3 даты из файла
    try:
        datetime = my_image.datetime
        # переводим даты в формат времени по шаблону выше
        datetime = pd.to_datetime(
                    fpattern
                        .findall(datetime)[0][0]
                        .replace(':','-') + ' ' + fpattern
                        .findall(datetime)[0][1]
                   )
    except:
        datetime = np.nan
    
    try:
        datetime_original = my_image.datetime_original
        datetime_original = pd.to_datetime(
                    fpattern
                        .findall(datetime_original)[0][0]
                        .replace(':','-') + ' ' + fpattern
                        .findall(datetime_original)[0][1]
                   )
    except:
        datetime_original = np.nan
    
    try:
        datetime_digitized = my_image.datetime_digitized
        datetime_digitized = pd.to_datetime(
                    fpattern
                        .findall(datetime_digitized)[0][0]
                        .replace(':','-') + ' ' + fpattern
                        .findall(datetime_digitized)[0][1]
                    )
    except:
        datetime_digitized = np.nan
        
    return datetime, datetime_original, datetime_digitized

In [19]:
def get_video_exif_date(full_path):
    """
    функция получает на вход полный путь к файлу и возвращает дату создания
    """
    media_info = MediaInfo.parse(full_path)
    for track in media_info.tracks:
        if track.track_type == "Video":
            try:
                exif_vdata = track.to_data()
                vdata = exif_vdata['tagged_date']

                # часовой пояс
                time_zone = pd.to_timedelta(7, unit='h')

                vdata = pd.to_datetime(vdata[4:]) + time_zone
                return vdata
            except:
                return np.nan

In [5]:
# ищем год в названии файла
def get_year_name(full_path):
    pattern = re.compile('(20\d\d)[^0-9]')
    years = pattern.findall(full_path)
    if len(years) > 0:
        return ' '.join(years)
    else:
        return np.nan

In [6]:
# извлекаем дату из json файла
def get_json_date(full_path):
    try:
        json_path = str(full_path) + '.json'
        with open(json_path) as json_file:
            json_data = json.load(json_file)
        
        # часовой пояс
        time_zone = pd.to_timedelta(7, unit='h')
        
        # извлекаем время из json, два варианта
        video_creation_time = pd.to_datetime(json_data["creationTime"]['timestamp'], unit='s') + time_zone
        video_taken_time    = pd.to_datetime(json_data["photoTakenTime"]["timestamp"], unit='s') + time_zone
        return video_creation_time, video_taken_time
    except:
        return np.nan, np.nan


In [19]:
%%time
# сначала получим весь список файлов, расшерений и путей
df = get_file_table(path)

Wall time: 5min 9s


In [20]:
df.extension.value_counts()

.json    31740
.jpg     28857
.mp4      2132
.mts       516
.jpeg       56
.avi        41
.nef        40
.mov        38
.3gp        19
.gif        13
.png         6
.mpg         3
.bmp         3
.mkv         1
Name: extension, dtype: int64

In [ ]:
ext_list = ['.jpeg', '.avi', '.nef', '.mov', '.3gp', '.gif', '.png', '.mpg', '.mpg', '.bmp', '.mkv']
base_goal_patch = 'G:/'
for x in ext_list:
    for file_path in  df.query('extension == @x').path:
        if not os.path.exists('DXF'):                                           # создаём папку DXF, если она не существует
            os.makedirs(x[1:])

            
            
# доделать....

In [25]:
ext = '.jpeg'
goal_patch = r'G:\jpeg'
for file_path in  df.query('extension == @ext').path:
    copy2(file_path
          , goal_patch +'/' + file_path.stem + ext)

In [ ]:
ext = '.avi'
goal_patch = r'G:\avi'
for file_path in  df.query('extension == @ext').path:
    copy2(file_path
          , goal_patch +'/' + file_path.stem + ext)

In [26]:
ext = '.nef'
goal_patch = r'G:\nef'
for file_path in  df.query('extension == @ext').path:
    copy2(file_path
          , goal_patch +'/' + file_path.stem + ext)

In [27]:
ext = '.mov'
goal_patch = r'G:\mov'
for file_path in  df.query('extension == @ext').path:
    copy2(file_path
          , goal_patch +'/' + file_path.stem + ext)

In [28]:
ext = '.3gp'
goal_patch = r'G:\3gp'
for file_path in  df.query('extension == @ext').path:
    copy2(file_path
          , goal_patch +'/' + file_path.stem + ext)

In [39]:
# списки расширений для видео и фото
# video_ext = np.array(['.mp4', '.mts', '.avi', '.mov', '.3gp', '.mpg', '.mkv'])
# photo_ext = np.array(['.jpg', '.jpeg', '.nef', '.gif', '.png', '.bmp'])

video_ext = np.array(['.mp4', '.mts'])
photo_ext = np.array(['.jpg', '.jpeg'])

In [40]:
%%time
df = pd.DataFrame(columns=['name', 'extension', 'path', 
                           'video_exif_date', 
                           'photo_date', 'photo_date_orig', 'photo_date_dig', 
                           'json_creation_time', 'json_taken_time',
                           'year_from_name'])

for file_path in path.glob('**/*.*'):
    # условие, чтобы элемент был файлом с расшерением видео или фото
    if file_path.is_file() and (file_path.suffix.lower() in video_ext or file_path.suffix.lower() in photo_ext):
        
        # обнуляем все значения, чтобы не записывались старые
        video_exif_date, photo_date, photo_date_orig, photo_date_dig, json_creation_time, json_taken_time, year_from_name = (np.nan,) * 7
        
        # имя файла
        name = file_path.stem
        # расшерение файла
        extension = file_path.suffix.lower()
        
        # если видео
        if file_path.suffix.lower() in video_ext:
            video_exif_date = get_video_exif_date(file_path)
#             # из json
#             json_creation_time, json_taken_time = get_json_date(file_path)
#             # из названия файла
#             year_from_name = get_year_name(name)
            
        # если фото
        elif file_path.suffix.lower() in photo_ext:
            photo_date, photo_date_orig, photo_date_dig = get_photo_exif_date(file_path)
        
        # из json
        json_creation_time, json_taken_time = get_json_date(file_path)
        
        # из названия файла
        year_from_name = get_year_name(name)

        df.loc[len(df)] = [name, extension, file_path, 
                           video_exif_date, 
                           photo_date, photo_date_orig, photo_date_dig, 
                           json_creation_time, json_taken_time,
                           year_from_name]
df['video_exif_date'] = pd.to_datetime(df['video_exif_date'])

<ipython-input-33-b0b58f583047>:28: RuntimeWarning: ASCII tag contains -1 fewer bytes than specified
  datetime_original = my_image.datetime_original


Wall time: 21min 48s


In [42]:
df.shape

(31561, 10)

In [43]:
df.dtypes

name                          object
extension                     object
path                          object
video_exif_date       datetime64[ns]
photo_date            datetime64[ns]
photo_date_orig       datetime64[ns]
photo_date_dig        datetime64[ns]
json_creation_time    datetime64[ns]
json_taken_time       datetime64[ns]
year_from_name                object
dtype: object

In [44]:
df.dropna(subset=['year_from_name'])

,name,extension,path,video_exif_date,photo_date,photo_date_orig,photo_date_dig,json_creation_time,json_taken_time,year_from_name
537,IMG_2002 copy,.jpg,G:\Google Photo_extr\Takeout\Google Фото\Photo...,NaT,2007-08-01 20:33:50,2007-07-24 15:42:10,2007-07-24 15:42:10,2016-07-24 11:26:06,2007-07-24 14:42:10,2002
553,IMG_2021 copy 2,.jpg,G:\Google Photo_extr\Takeout\Google Фото\Photo...,NaT,2007-08-01 20:18:28,2007-07-24 16:06:07,2007-07-24 16:06:07,2016-07-24 11:26:16,2007-07-24 15:06:07,2021
554,IMG_2021 copy,.jpg,G:\Google Photo_extr\Takeout\Google Фото\Photo...,NaT,2007-08-01 20:18:35,2007-07-24 16:06:07,2007-07-24 16:06:07,2016-07-24 11:26:17,2007-07-24 15:06:07,2021
572,IMG_2049 copy,.jpg,G:\Google Photo_extr\Takeout\Google Фото\Photo...,NaT,2007-08-01 20:56:48,2007-07-25 13:59:58,2007-07-25 13:59:58,2016-07-24 11:26:29,2007-07-25 12:59:58,2049
575,IMG_2051 copy,.jpg,G:\Google Photo_extr\Takeout\Google Фото\Photo...,NaT,2007-08-02 13:56:54,2007-07-25 21:49:54,2007-07-25 21:49:54,2016-07-24 11:26:31,2007-07-25 20:49:54,2051
...,...,...,...,...,...,...,...,...,...,...
29502,IMG_2066 copy,.jpg,G:\Google Photo_extr\Takeout\Google Фото\Поезд...,NaT,2007-08-01 22:47:04,2007-07-25 22:38:00,2007-07-25 22:38:00,2016-07-24 11:35:12,2007-07-25 21:38:00,2066
29504,IMG_2067 copy,.jpg,G:\Google Photo_extr\Takeout\Google Фото\Поезд...,NaT,2007-08-01 22:46:09,2007-07-25 22:38:07,2007-07-25 22:38:07,2016-07-24 11:35:12,2007-07-25 21:38:07,2067
29523,IMG_2087 copy,.jpg,G:\Google Photo_extr\Takeout\Google Фото\Поезд...,NaT,2007-08-01 22:33:52,2007-07-26 10:17:29,2007-07-26 10:17:29,2016-07-24 11:35:12,2007-07-26 09:17:29,2087
29527,IMG_2092 copy,.jpg,G:\Google Photo_extr\Takeout\Google Фото\Поезд...,NaT,2007-08-01 22:31:24,2007-07-26 10:19:46,2007-07-26 10:19:46,2016-07-24 11:35:12,2007-07-26 09:19:46,2092


In [72]:
# список всех методов
# my_image.list_all()